<a href="https://colab.research.google.com/github/aminaakm/amina/blob/main/CART_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving cart_py_data.csv to cart_py_data.csv
User uploaded file "cart_py_data.csv" with length 54726090 bytes


In [2]:
# 1. Setup and Data Loading
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression # Although the paper uses CART, Logistic Regression is needed for the null model deviance calculation.
from sklearn.metrics import log_loss, accuracy_score
import matplotlib.pyplot as plt
import io

# Assuming the user has uploaded the file named 'nfhs_data.csv'
# You can use the files.upload() method if the file is not already present.
# For demonstration, let's assume the data is already uploaded and read it.
# If you need to upload, uncomment the following lines:
# from google.colab import files
# uploaded = files.upload()
# file_name = list(uploaded.keys())[0]
# df = pd.read_csv(io.BytesIO(uploaded[file_name]))

# Replace with the actual file name if different
file_name = 'cart_py_data.csv' # Assuming the previously uploaded file is used
df = pd.read_csv(file_name)

print("Original DataFrame head:")
display(df.head())
print("\nOriginal DataFrame Info:")
df.info()

/tmp/ipython-input-2280030768.py:23: DtypeWarning: Columns (0,2,4,5,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


Original DataFrame head:


,caseid,sbp_avg,sex,age,height,weight,bmi,WC,HC,WHR
0,0100101305 04,123.333333,male,22.0,142.7,35.3,17.33511628,101.4,115.9,0.874892148403796
1,0100101305 05,118.000000,male,19.0,139.7,31,15.88432929,97.3,112.6,0.864120781527531
2,0100101345 02,114.333333,male,38.0,147.9,75.5,34.51521664,121.7,128.6,0.946345256609642
3,0100101383 02,119.333333,male,40.0,162.7,70.8,26.7459576,109.3,120.7,0.905550952775476
4,0100101383 03,121.333333,male,22.0,152.7,35.8,15.35341371,109.0,100,1.09



Original DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724115 entries, 0 to 724114
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   caseid   646651 non-null  object 
 1   sbp_avg  646651 non-null  float64
 2   sex      646651 non-null  object 
 3   age      646651 non-null  float64
 4   height   646651 non-null  object 
 5   weight   646651 non-null  object 
 6   bmi      646651 non-null  object 
 7   WC       646651 non-null  float64
 8   HC       707655 non-null  object 
 9   WHR      646706 non-null  object 
dtypes: float64(3), object(7)
memory usage: 55.2+ MB


In [3]:
# 2. Data Preprocessing

# Clean column names by stripping whitespace
df.columns = df.columns.str.strip()

# Convert relevant columns to numeric, coercing errors
numeric_cols = ['sbp_avg', 'age', 'height', 'weight', 'bmi', 'WC', 'HC', 'WHR']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with missing values in the relevant columns for this analysis
df.dropna(subset=['sex'] + numeric_cols, inplace=True)

# Create separate DataFrames for males and females
df_men = df[df['sex'].str.lower() == 'male'].copy()
df_women = df[df['sex'].str.lower() == 'female'].copy()

# Define the binary target variable 'increased_bp'
# For df_women, increased_bp is 1 if sbp_avg > 120, and 0 otherwise.
df_women['increased_bp'] = (df_women['sbp_avg'] > 120).astype(int)

# For df_men, increased_bp is 1 if sbp_avg > 140, and 0 otherwise.
df_men['increased_bp'] = (df_men['sbp_avg'] > 140).astype(int)

# Define the list of predictor features
features = ['bmi', 'WC', 'HC', 'WHR']

# Split both df_men and df_women into training and testing sets
X_train_men, X_test_men, y_train_men, y_test_men = train_test_split(
    df_men[features], df_men['increased_bp'], test_size=0.2, random_state=42
)

X_train_women, X_test_women, y_train_women, y_test_women = train_test_split(
    df_women[features], df_women['increased_bp'], test_size=0.2, random_state=42
)

print("\nProcessed DataFrames head (Men):")
display(X_train_men.head())
print("\nProcessed DataFrames head (Women):")
display(X_train_women.head())


Processed DataFrames head (Men):


,bmi,WC,HC,WHR
305689,18.702469,63.8,97.6,0.653689
471201,24.747246,74.3,81.6,0.910539
390770,28.791430,81.9,91.1,0.899012
631672,21.295958,69.2,76.5,0.904575
515928,23.641125,69.2,105.0,0.659048



Processed DataFrames head (Women):


,bmi,WC,HC,WHR
84139,27.577948,105.5,97.8,1.078732
151411,22.049475,79.0,102.3,0.772239
572262,18.916740,76.3,60.4,1.263245
301181,21.555556,68.2,85.8,0.794872
232396,20.195286,73.1,100.6,0.726640


In [9]:
# 3. Classification Tree (CART) Analysis (Replicating Table 2)

def calculate_metrics(model, X, y):
    """Calculates deviance, misclassification error, and pseudo R^2."""
    y_pred_proba = model.predict_proba(X)[:, 1]
    deviance_model = log_loss(y, y_pred_proba, normalize=False)

    y_pred = model.predict(X)
    misclassification_error = 1 - accuracy_score(y, y_pred)

    # Calculate null deviance
    # Check if there is more than one class in the target variable
    if len(y.unique()) > 1:
        # Calculate null deviance using a Logistic Regression model predicting the majority class
        # This approach requires at least two samples of each class.
        # If that's not met, the alternative calculation below will be used.
        if y.value_counts().min() >= 2:
            try:
                null_model = LogisticRegression(solver='liblinear') # Using LogisticRegression for null model
                # Create a dummy target array with the majority class for fitting the null model
                dummy_y = [y.mode()[0]] * len(y)
                # Fit the null model - X is used here just to satisfy the fit method's requirement for input features
                # The predictions will be based solely on the intercept
                null_model.fit(X, dummy_y)
                y_pred_proba_null = null_model.predict_proba(X)[:, 1]
                deviance_null = log_loss(y, y_pred_proba_null, normalize=False)
                pseudo_r2 = 1 - (deviance_model / deviance_null) if deviance_null != 0 else np.nan
            except ValueError:
                 # Fallback if LogisticRegression still fails to fit even with the check
                 majority_class = y.mode()[0]
                 proportion_majority = (y == majority_class).sum() / len(y)
                 if proportion_majority > 0 and proportion_majority < 1:
                      deviance_null = -2 * np.sum(y * np.log(proportion_majority) + (1 - y) * np.log(1 - proportion_majority))
                      pseudo_r2 = 1 - (deviance_model / deviance_null) if deviance_null != 0 else np.nan
                 else:
                      deviance_null = 0
                      pseudo_r2 = np.nan
        else:
            # If there are unique values but not enough samples for each class for LR,
            # calculate null deviance based on predicting the proportion of the majority class
            majority_class = y.mode()[0]
            proportion_majority = (y == majority_class).sum() / len(y)
            # log_loss for a constant prediction of the majority class proportion
            if proportion_majority > 0 and proportion_majority < 1: # Avoid log(0) or log(1)
                 deviance_null = -2 * np.sum(y * np.log(proportion_majority) + (1 - y) * np.log(1 - proportion_majority))
                 pseudo_r2 = 1 - (deviance_model / deviance_null) if deviance_null != 0 else np.nan
            else:
                 # If all instances belong to the same class, the null deviance is effectively 0 or undefined
                 deviance_null = 0
                 pseudo_r2 = np.nan # Or 0, depending on the desired behavior when null deviance is 0

    else:
        # If only one class in the target variable, calculate null deviance based on predicting that class's proportion
        majority_class = y.mode()[0]
        proportion_majority = (y == majority_class).sum() / len(y) # This will be 1.0
        # log_loss for a constant prediction of the single existing class (proportion 1.0)
        # This will be 0 if y is all 0s and we predict 0, or all 1s and we predict 1.
        # It will be undefined/infinity if y is all 0s and we predict 1, or all 1s and we predict 0.
        # Given how the null model is defined (predicting the majority class), this case should result in 0 deviance.
        deviance_null = 0
        pseudo_r2 = np.nan # Pseudo R^2 is not meaningful when null deviance is 0


    return deviance_model, misclassification_error, pseudo_r2

# Store results
results_men = []
results_women = []

# Iterate through all possible combinations of features
for i in range(1, len(features) + 1):
    for combo in itertools.combinations(features, i):
        current_features = list(combo)

        # Men's data
        X_train_men_subset = X_train_men[current_features]
        cart_men = DecisionTreeClassifier(random_state=42)
        cart_men.fit(X_train_men_subset, y_train_men)
        deviance_men, misclassification_men, pseudo_r2_men = calculate_metrics(cart_men, X_train_men_subset, y_train_men)
        results_men.append({
            'Predictors': ', '.join(current_features),
            'Deviance': deviance_men,
            'Misclassification Error Rate': misclassification_men,
            'Pseudo R^2': pseudo_r2_men
        })

        # Women's data
        X_train_women_subset = X_train_women[current_features]
        cart_women = DecisionTreeClassifier(random_state=42)
        cart_women.fit(X_train_women_subset, y_train_women)
        deviance_women, misclassification_women, pseudo_r2_women = calculate_metrics(cart_women, X_train_women_subset, y_train_women)
        results_women.append({
            'Predictors': ', '.join(current_features),
            'Deviance': deviance_women,
            'Misclassification Error Rate': misclassification_women,
            'Pseudo R^2': pseudo_r2_women
        })

# Create DataFrames from results
results_df_men = pd.DataFrame(results_men)
results_df_women = pd.DataFrame(results_women)

print("\nCART Analysis Results (Men - Training Data):")
display(results_df_men.sort_values(by='Pseudo R^2', ascending=False))

print("\nCART Analysis Results (Women - Training Data):")
display(results_df_women.sort_values(by='Pseudo R^2', ascending=False))

# Identify the best model based on Pseudo R^2
best_model_men = results_df_men.loc[results_df_men['Pseudo R^2'].idxmax()]
best_model_women = results_df_women.loc[results_df_women['Pseudo R^2'].idxmax()]

print("\nBest CART Model (Men - Training Data) based on Pseudo R^2:")
display(best_model_men)

print("\nBest CART Model (Women - Training Data) based on Pseudo R^2:")
display(best_model_women)


CART Analysis Results (Men - Training Data):


,Predictors,Deviance,Misclassification Error Rate,Pseudo R^2
6,"bmi, WHR",2.772589,0.000005,0.999999
14,"bmi, WC, HC, WHR",2.772589,0.000005,0.999999
10,"bmi, WC, HC",2.772589,0.000005,0.999999
11,"bmi, WC, WHR",2.772589,0.000005,0.999999
12,"bmi, HC, WHR",2.772589,0.000005,0.999999
5,"bmi, HC",565.635201,0.000957,0.999759
4,"bmi, WC",906.832047,0.001519,0.999613
9,"HC, WHR",37316.334847,0.040163,0.984088
8,"WC, WHR",37316.334847,0.040163,0.984088
7,"WC, HC",37316.334847,0.040163,0.984088



CART Analysis Results (Women - Training Data):


,Predictors,Deviance,Misclassification Error Rate,Pseudo R^2
6,"bmi, WHR",1.386294,0.000012,0.999989
14,"bmi, WC, HC, WHR",1.386294,0.000012,0.999989
10,"bmi, WC, HC",1.386294,0.000012,0.999989
11,"bmi, WC, WHR",1.386294,0.000012,0.999989
12,"bmi, HC, WHR",1.386294,0.000012,0.999989
5,"bmi, HC",101.199488,0.000895,0.999233
4,"bmi, WC",184.377150,0.001630,0.998602
9,"HC, WHR",13035.471806,0.100283,0.901154
8,"WC, WHR",13035.471806,0.100283,0.901154
7,"WC, HC",13035.471806,0.100283,0.901154



Best CART Model (Men - Training Data) based on Pseudo R^2:


,6
Predictors,"bmi, WHR"
Deviance,2.772589
Misclassification Error Rate,0.000005
Pseudo R^2,0.999999



Best CART Model (Women - Training Data) based on Pseudo R^2:


,6
Predictors,"bmi, WHR"
Deviance,1.386294
Misclassification Error Rate,0.000012
Pseudo R^2,0.999989
